# HTTP

## Architecture
```
         HTTP Request
   > > > > > > > > > > > > 
  |                      |
Client                 Server
  |                      |
   < < < < < < < < < < < <
         HTTP Response
```
- The response **MUST** be sent even if Error occurs
- Proper **REQUEST HEADER** should be set manually

## How to check whether legal or illegal
- Everything in DB are private
- Check ```robots.txt```
- Syntax(just convention)
```
# Welcom to my robots.txt file! <--- comment
User-agent: * # * is wild card
Allow: *
Disallow: /private # Except /private, can access all

User-agent: Googlebot
Allow: /?_escaped_fragment_
```
- Permission is overlapped


In [53]:
# Parsing robot.txt
from urllib.robotparser import RobotFileParser

parser = RobotFileParser(r'https://www.google.com/robots.txt')
parser.read()
for entry in parser.entries:
    print(entry.useragents)
    for rule in entry.rulelines:
        print(rule.allowance, rule.path)
    print("================")

defaultEntry = parser.default_entry
print("*")
for rule in defaultEntry.rulelines:
    print(rule.allowance, rule.path)
    print("================")

['AdsBot-Google']
False /maps/api/js/
True /maps/api/js
False /maps/api/place/js/
False /maps/api/staticmap
False /maps/api/streetview
['Twitterbot']
True /imgres
['facebookexternalhit']
True /imgres
*
False /search
True /search/about
True /search/static
True /search/howsearchworks
False /sdch
False /groups
False /index.html
False /
True /%3Fhl%3D
False /%3Fhl%3D%2A%26
True /%3Fhl%3D%2A%26gws_rd%3Dssl%24
False /%3Fhl%3D%2A%26%2A%26gws_rd%3Dssl
True /%3Fgws_rd%3Dssl%24
True /%3Fpt1%3Dtrue%24
False /imgres
False /u/
False /preferences
False /setprefs
False /default
False /m
False /m/
True /m/finance
False /wml
False /wml/
False /wml/search
False /xhtml
False /xhtml/
False /xhtml/search
False /xml
False /imode
False /imode/
False /imode/search
False /jsky
False /jsky/
False /jsky/search
False /pda
False /pda/
False /pda/search
False /sprint_xhtml
False /sprint_wml
False /pqa
False /palm
False /gwt/
False /purchases
False /local
False /local_url
False /shihui
False /shihui/
False /products

## HTTP S

In [ ]:
import requests

url = 'https://www.google.com'
doc = requests.get(url, )

In [ ]:
quote quote_plus
httpbin.org